# Thu thập dữ liệu cổ phiếu Amazon từ` trang web
https://finance.yahoo.com/quote/AMZN/history?period1=1495324800&period2=1653091200&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import csv

In [33]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
url = "https://finance.yahoo.com/quote/AMZN/history?period1=1495324800&period2=1653091200&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"

In [34]:
# r = requests.get(url, headers=headers)
# soup = BeautifulSoup(r.text, 'html')
# tr_rows = soup.find_all('tr')[1]
# print(tr_rows)
# list_date = []
# list_open = []
# list_high = []
# list_low = []
# list_close = []
# list_adj_close = []
# list_volume = []
# list_all = [list_date,list_open,list_high,list_low,list_close,list_adj_close,list_volume]
# # list_all[0].append(1)
# # print(list_all)
# for tr_row in tr_rows:
#     i = 0
#     td_columns = tr_row.select('td')
#     for td_column in td_columns:
#         list_all[i].append(td_column.getText())
#         i+=1
        

# print(len(list_all[1]))


In [35]:
stock = 'AMZN'
stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}'

params = {
    'range':'5y',
    'interval': '1d',
    'events':'history',
    'includeAdjustedClose':'true'
}
response = requests.get(stock_url.format(stock),params=params,headers=headers)

`Lưu dữ liệu thu thập được vào file dữ liệu thô có tên là : raw_data.csv`

In [55]:
file  = StringIO(response.text)
reader = csv.reader(file)
data = list(reader)

with open('raw data/raw_data.csv', 'w',newline='') as f:
    writer = csv.writer(f)
    # writer.writerow(("Date", "Open","High", "Low","Close", "Adj Close","Volume"))
    for row in data:
        writer.writerow(row)
